In [1]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 33.3 MB/s eta 0:00:00


In [2]:
!pip install brotli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.6 MB/s eta 0:00:00


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json

In [4]:
# here we mounted google drive so all the extracted data will saved on drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
drive_path = '/content/drive/MyDrive/Myscheme_data'

In [6]:
# This function goes all pages and extract slug values that used in individual scheme page link and return final set of links  (e.g. https://www.myscheme.gov.in/schemes/pmsby)

import brotli
import urllib.parse

def fetch_and_extract_slugs(from_index, size):
    url = "https://api.myscheme.gov.in/search/v4/schemes"
    params = {
        "lang": "en",
        "q": "[]",
        "keyword": "",
        "sort": "",
        "from": str(from_index),
        "size": str(size),
    }
    headers = {
        "Accept": "application/json, text/plain, */*",
        "Accept-Encoding": "gzip, deflate, br",
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/135.0.0.0 Safari/537.36"
        ),
        "Origin": "https://www.myscheme.gov.in",
        "x-api-key": "tYTy5eEhlu9rFjyxuCr7ra7ACp4dv1RH8gWuHTDc",
    }

    # 1. Fetch
    resp = requests.get(url, params=params, headers=headers)
    resp.raise_for_status()

    # 2. Decompress if needed
    raw = resp.content
    encoding = resp.headers.get("Content-Encoding", "")
    if "br" in encoding:
        try:
            raw = brotli.decompress(raw)
        except brotli.error:
            # if decompression fails, keep raw bytes
            pass

    # 3. Parse JSON
    data = json.loads(raw.decode("utf-8"))

    # 4. Extract slugs
    slugs = []
    for item in data.get("data", {}) \
                   .get("hits", {}) \
                   .get("items", []):
        # safe‑get slug from fields
        slug = item.get("fields", {}).get("slug")
        if slug is not None:
          full_url = urllib.parse.urljoin("https://www.myscheme.gov.in/schemes/", slug)

          slugs.append(full_url)

    return slugs


In [ ]:
# This function take link of scheme page (e.g. - https://www.myscheme.gov.in/schemes/pmsby)

In [7]:
from bs4 import BeautifulSoup
import requests
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

def extract_scheme_details(link, headless=True):
    headers = {
        "Host": "www.myscheme.gov.in",
        "Accept": "application/json, text/plain, */*",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,mr;q=0.6",
        "Cookie": "_ga=GA1.1.1202555016.1744885791; _ga_GG57J0N1P4=GS1.1.1744911857.6.1.1744911863.0.0.0",
        "Priority": "u=1, i",
        "Referer": "https://www.myscheme.gov.in/search",
        "Sec-Ch-Ua": '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": '"Windows"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
    }

    # Configure Chrome options
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--pageLoadStrategy=none")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument(f"user-agent={headers['User-Agent']}")

    # Start Chrome browser with options
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(link)

    time.sleep(3)

    # Get the page source and parse it with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    # Find the JSON blob
    script_tag = soup.find('script', id='__NEXT_DATA__')
    if not script_tag or not script_tag.string:
        return {"error": "JSON data not found on page"}

    # Parse JSON
    data = json.loads(script_tag.string)
    props = data.get('props', {}).get('pageProps', {})
    scheme_data = props.get('schemeData', {}).get('en', {})
    docs_data = props.get('docs', {}).get('data', {}).get('en', {})

    # Safe extraction helpers
    def safe_label(obj, key):
        """Return obj[key]['label'] if it exists and is not None, else ''."""
        val = obj.get(key)
        return val.get('label') if isinstance(val, dict) and val.get('label') else ""

    structured_data = {
        "Scheme Name": scheme_data.get('basicDetails', {}).get('schemeName', ""),
        "Ministries/Departments/State": {
            "State": safe_label(scheme_data.get('basicDetails', {}), 'state'),
            "Department": safe_label(scheme_data.get('basicDetails', {}), 'nodalDepartmentName')
        },
        "Eligibility": scheme_data.get('eligibilityCriteria', {}).get('eligibilityDescription_md', ""),
        "Details": scheme_data.get('schemeContent', {}).get('detailedDescription_md', ""),
        "Tags": scheme_data.get('basicDetails', {}).get('tags', []),
        "Application Process": [],
    }

    apps = []
    for div in soup.find_all("div", class_="rounded py-4 px-4"):
        apps.append(div.get_text(" ", strip=True))
    structured_data["Application Process"].append(apps)

    sections = {
        'benefits': 'Benefits',
        'documents-required': 'Documents Required'
    }

    for section_id, section_name in sections.items():
        section_div = soup.find('div', id=section_id)
        if section_div:
            markdown_div = section_div.find('div', class_='markdown-options')
            if markdown_div:
                full_text = markdown_div.get_text(separator='\n', strip=True)
                lines = full_text.split('\n')
                # Remove the first line if it matches the section name (case-insensitive)
                if lines and lines[0].lower() == section_name.lower().replace('-', ' '):
                    content = '\n'.join(lines[1:])
                else:
                    content = full_text
                structured_data[section_name] = content

    return structured_data

In [ ]:
import os
scheme_link_list = fetch_and_extract_slugs(from_index=0, size=10)

In [9]:
# Scraping data one page at time

output_file = os.path.join(drive_path, 'schemes_data.csv')


records = []
for link in scheme_link_list:
    print(f"Processing {link} …")
    try:
        data = extract_scheme_details(link, headless=True)
        data['Source URL'] = link
        records.append(data)
    except Exception as e:
        print(f"  ⚠️ Error on {link}: {e}")

# 3) Build DataFrame of new records
new_df = pd.DataFrame(records)

# 4) If CSV exists, read and append; otherwise, create new
if os.path.exists(output_file):
    existing_df = pd.read_csv(output_file)

    combined = pd.concat([existing_df, new_df], ignore_index=True)

    combined.drop_duplicates(subset=['Source URL'], inplace=True)
    combined.to_csv(output_file, index=False, encoding='utf-8')
    print(f"✅ Appended {len(new_df)} records, now total {len(combined)} saved to {output_file}")
else:
    # create new file
    new_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"✅ Created new table with {len(new_df)} records at {output_file}")

In [14]:
# verification
df = pd.read_csv("/content/drive/MyDrive/Myscheme_data/schemes_data.csv")

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3323 entries, 0 to 3322
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Scheme Name                   3313 non-null   object
 1   Ministries/Departments/State  3313 non-null   object
 2   Eligibility                   3313 non-null   object
 3   Details                       3313 non-null   object
 4   Tags                          3313 non-null   object
 5   Application Process           3313 non-null   object
 6   Benefits                      3313 non-null   object
 7   Documents Required            3311 non-null   object
 8   Source URL                    3323 non-null   object
 9   error                         10 non-null     object
dtypes: object(10)
memory usage: 259.7+ KB


In [12]:
df.tail(5)

,Scheme Name,Ministries/Departments/State,Eligibility,Details,Tags,Application Process,Benefits,Documents Required,Source URL,error
3318,Distribution of Female Menstrual Hygiene Kit f...,{'State': 'Dadra & Nagar Haveli and Daman & Di...,&gt; **For Registration as a Building/ Constru...,The &amp;quot;Distribution of Female Menstrual...,"['Building Worker', 'Construction Worker', 'Fe...",[['Registration Process as a Building/ Constru...,Distribution of Menstrual Hygiene Kits to the ...,For Registration as a Building/ Construction W...,https://www.myscheme.gov.in/schemes/dfmkftcw,NaN
3319,Medical Assistance and Wage Loss for the Const...,{'State': 'Dadra & Nagar Haveli and Daman & Di...,&gt; **For Registration as a Building/ Constru...,The &amp;quot;Medical Assistance and Wage Loss...,"['Building Worker', 'Construction Worker', 'La...",[['Registration Process as a Building/ Constru...,Reimbursement of Medical Expenses:\nThe Board ...,For Registration as a Building/ Construction W...,https://www.myscheme.gov.in/schemes/mawlftcw,NaN
3320,Self Education for the Construction Workers,{'State': 'Dadra & Nagar Haveli and Daman & Di...,&gt; **For Registration as a Building/ Constru...,The &amp;quot;Skill Development for the Constr...,"['Building Worker', 'Building Worker', 'Constr...",[['Registration Process as a Building/ Constru...,Reimbursement :\nCourse fees for distance educ...,For Registration as a Building/ Construction W...,https://www.myscheme.gov.in/schemes/seftcw,NaN
3321,Payment of Funeral Assistance for the Construc...,{'State': 'Dadra & Nagar Haveli and Daman & Di...,&gt; **For Registration as a Building/ Constru...,The &amp;quot;Payment of Funeral Assistance fo...,"['Building Worker', 'Construction Worker', 'De...",[['Registration Process as a Building/ Constru...,"Quantum of Assistance:\n₹20,000/-.",For Registration as a Building/ Construction W...,https://www.myscheme.gov.in/schemes/pofaftcw,NaN
3322,Grant of Token Amount for Construction of Hous...,{'State': 'Dadra & Nagar Haveli and Daman & Di...,&gt; **For Registration as a Building/ Constru...,The &amp;quot;Grant of Token Amount for Constr...,"['Building Workers', 'Construction Of House', ...",[['Registration Process as a Building/ Constru...,"Token Amount:\n₹75,000/-.\nFor applicants cove...",For Registration as a Building/ Construction W...,https://www.myscheme.gov.in/schemes/gtachcw,NaN


In [13]:
# Loop for many pages at time

# === Configuration ===
drive_path     = "/content/drive/MyDrive/Myscheme_data"     # adjust to your drive mount
output_file    = os.path.join(drive_path, "schemes_data.csv")
initial_index  = 900
final_index    = 909
batch_size     = 10
sleep_secs     = 50

def main():
    from_index = initial_index

    while from_index <= final_index:
        # determine how many to fetch in this batch
        remaining = final_index - from_index + 1
        size_to_fetch = min(batch_size, remaining)

        print(f"\n▶️  Fetching slugs from index {from_index} (size={size_to_fetch})…")
        slugs = fetch_and_extract_slugs(from_index, size_to_fetch)
        if not slugs:
            print("🔚  No slugs returned; exiting early.")
            break

        records = []
        for link in slugs:
            print(f"   • Processing {link} …")
            try:
                data = extract_scheme_details(link, headless=True)
                data['Source URL'] = link
                records.append(data)
            except Exception as e:
                print(f"     ⚠️  Error on {link}: {e}")

        new_df = pd.DataFrame(records)

        # Append or create CSV
        if os.path.exists(output_file):
            existing_df = pd.read_csv(output_file)
            combined    = pd.concat([existing_df, new_df], ignore_index=True)
            combined.drop_duplicates(subset=['Source URL'], inplace=True)
            combined.to_csv(output_file, index=False, encoding='utf-8')
            print(f"✅  Appended {len(new_df)} records; total now {len(combined)} rows.")
        else:
            os.makedirs(drive_path, exist_ok=True)
            new_df.to_csv(output_file, index=False, encoding='utf-8')
            print(f"✅  Created new CSV with {len(new_df)} records at {output_file}")

        from_index += size_to_fetch
        if from_index <= final_index:
            print(f"⏱️  Sleeping for {sleep_secs} seconds before next batch…")
            time.sleep(sleep_secs)

    print("✔️  Completed fetching from "
          f"{initial_index} through {final_index}.")

if __name__ == "__main__":
    main()



▶️  Fetching slugs from index 900 (size=10)…
   • Processing https://www.myscheme.gov.in/schemes/haritha-haram …
   • Processing https://www.myscheme.gov.in/schemes/dk …
   • Processing https://www.myscheme.gov.in/schemes/oadp …
   • Processing https://www.myscheme.gov.in/schemes/yipb …
   • Processing https://www.myscheme.gov.in/schemes/dpps …
   • Processing https://www.myscheme.gov.in/schemes/naospasa-bpjmbpasa …
   • Processing https://www.myscheme.gov.in/schemes/serb-star …
   • Processing https://www.myscheme.gov.in/schemes/ds …
   • Processing https://www.myscheme.gov.in/schemes/stccbocwwb …
   • Processing https://www.myscheme.gov.in/schemes/rtftsccw …
✅  Appended 10 records; total now 3323 rows.
✔️  Completed fetching from 900 through 909.
